In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import math 

import tensorflow as tf
import tensorflow.keras as ks
from   tensorflow.keras import backend as Ks
import theano
from PIL import Image


WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


In [2]:
num_images = 100 
image_dim  = 400

working_type = tf.float16
image_type   = np.uint8

**I. Build Training Set **

In [3]:
def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def build_training(include_ground_truth = False):
    
    def concatenate_images(img, gt_img):
        nChannels = len(gt_img.shape)
        w = gt_img.shape[0]
        h = gt_img.shape[1]
        if nChannels == 3:
            cimg = np.concatenate((img, gt_img), axis=1)
        else:
            gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
            gt_img8 = img_float_to_uint8(gt_img)          
            gt_img_3c[:,:,0] = gt_img8
            gt_img_3c[:,:,1] = gt_img8
            gt_img_3c[:,:,2] = gt_img8
            img8 = img_float_to_uint8(img)
            cimg = np.concatenate((img8, gt_img_3c), axis=1)
        return cimg

    root_dir = "training/"
    image_dir = root_dir + "images/"
    files = os.listdir(image_dir)

    n = len(files) 
    imgs = [load_image(image_dir + files[i]) for i in range(n)]
    

    gt_dir = root_dir + "groundtruth/"
    gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
    
    return  np.array(imgs,dtype = np.float16)
    #return tf.data.Dataset.from_tensor_slices((imgs, gt_imgs))
    #return np.array([concatenate_images(imgs[0], gt_imgs[0]) for i in range(n)],dtype=np.uint8)
    


In [11]:
patch_shape          = (400,400,3)




window_size      = 15   # must be odd 
pool_size        = 4    # must be either 1,2,4,16
training_samples = int((image_dim/pool_size)**2*num_images)

training                  =  build_training()





rgb_input                 =  ks.layers.Input(shape=patch_shape, dtype='float16', name='rgb_input' )
rgb_pooled                =  ks.layers.AveragePooling2D(pool_size=(pool_size, pool_size)       , name='rgb_pooled') (rgb_input)
rgb_tiled                 =  ks.layers.Lambda(\
                                lambda img : tf.reshape(tf.image.extract_image_patches(img,
                                             [1,window_size,window_size,1],[1,1,1,1],[1,1,1,1],
                                             "SAME"),[-1,int(image_dim/pool_size),int(image_dim/pool_size),window_size,window_size,3]),
                                              name='rgb_tiled')(rgb_pooled)
rgb_resized               =  ks.layers.Lambda(\
                                lambda img: tf.reshape(img,[32*100*100,window_size,window_size,3]))(rgb_tiled)
    
model                     =  ks.Model(inputs=[rgb_input], outputs=[rgb_resized])


print(rgb_pooled.shape)
print(rgb_resized.shape)



features = model.predict(training)



(?, 100, 100, 3)
(320000, 15, 15, 3)


ValueError: could not broadcast input array from shape (320000,15,15,3) into shape (32,15,15,3)

In [ ]:
plt.imshow(features[1].astype(np.float32))
print(features.shape)

In [ ]:
plt.imshow(training[1].astype(np.float32))

**II.Setup Convolution **

In [ ]:
def normalize(input_tensor): 
    shape    = tf.shape(input_tensor)
    shape    = [shape[0],shape[1],shape[2],1]
    return tf.reshape(input_tensor/tf.reduce_max(tf.abs(input_tensor)),shape)

def greyscale(input_tensor): 
    return normalize(tf.reduce_mean(input_tensor,axis=3))

def blur(input_tensor,blur_width): 
    
    kernel = np.ones([blur_width,blur_width])
    with tf.name_scope('convolution'):
        conv     = tf.constant(kernel, dtype=working_type, shape=(blur_width, blur_width, 1, 1)) 
        output   = tf.nn.conv2d(input=input_tensor, filter=conv, strides=[1, 1, 1, 1], padding='SAME')
    
    return normalize(output)

def sobel(input_tensor): 

    kernel_h = np.array([3, 3])
    kernel_h = [ [1,2,1], [0,0,0], [-1,-2,-1] ]
    kernel_v = np.array([3, 3])
    kernel_v = [ [1,0,1], [2,0,-2], [-1,0,-1] ]

    with tf.name_scope('convolution'):
        conv_w_h = tf.constant(kernel_h, dtype=working_type, shape=(3, 3, 1, 1))
        conv_w_v = tf.constant(kernel_v, dtype=working_type, shape=(3, 3, 1, 1))    
        output_h = tf.nn.conv2d(input=input_tensor, filter=conv_w_h, strides=[1, 1, 1, 1], padding='SAME')
        output_v = tf.nn.conv2d(input=input_tensor, filter=conv_w_v, strides=[1, 1, 1, 1], padding='SAME')
        output   = (output_h**2 + output_v**2)**0.5        
    
    return normalize(output) 


def RGB(input_tensor, op): 
      
    shape    = tf.shape(input_tensor)
    shape    = [shape[0],shape[1],shape[2],1]
    
    R     = tf.cast(tf.reshape(input_tensor[:,:,:,0],shape),working_type)
    G     = tf.cast(tf.reshape(input_tensor[:,:,:,1],shape),working_type)
    B     = tf.cast(tf.reshape(input_tensor[:,:,:,2],shape),working_type) 
    
    return ((op(R)**2+op(G)**2+op(B)**2)**0.5)
    


    
    
    
input_placeholder = tf.placeholder( dtype=working_type, shape=(num_images, image_dim, image_dim, 3))
training    = build_training()
output_1    = RGB(input_placeholder,lambda x:sobel(blur(x,4)))
output_2    = sobel(blur(greyscale(input_placeholder),4))

result_1    = tf.Session().run(output_1, feed_dict={input_placeholder: training})
result_2    = tf.Session().run(output_2, feed_dict={input_placeholder: training})
    

In [ ]:
##
# https://medium.com/@tempflip/lane-detection-with-numpy-2-hough-transform-f4c017f4da39
##

def build_hough_lines(input_tensor,num_thetas,num_diams): 
    
    hough       = np.zeros((num_thetas,num_diams))
    theta_space = np.linspace(0, 2*math.pi, num_thetas)
    rad_space   = np.linspace(0,(img_shape/2)*(2**0.5),num_diams)
    
    for i,theta in enumerate(theta_space):
        for j, rad in enumerate(rad_space): 
            hough[i,j] = 
            
            x1 = image_dim/2 + diam*math.cos(theta)
            y1 = image_dim/2 + diam*math.sin(theta)
            
            x2 = 0 
            x2 = im
            
            
            dx = x2 - x1
            dy = y2 - y1
for x from x1 to x2 {
  y = y1 + dy * (x - x1) / dx
  plot(x, y)
}
    
def build_hough_space_fom_image(img, shape = (100, 300), val = 1):
    hough_space = np.zeros(shape)
    for i, row in enumerate(img):
        for j, pixel in enumerate(row):   
            if pixel != val : continue
            hough_space = add_to_hough_space_polar((i,j), hough_space)
    return hough_space

def add_to_hough_space_polar(p, feature_space):
    
    d_max = len(feature_space[0]) / 2
    for i in range(len(space)):
        theta = space[i]
        d = int(p[0] * math.sin(theta) + p[1] * math.cos(theta)) + d_max
        if (d >= d_max * 2) : continue
        feature_space[i, d] += 1
    return feature_space

In [ ]:
res = build_hough_space_fom_image(result_1[1].reshape(400,400), shape = (400, 400), val = 0)
plt.imshow(res.astype(np.float32),cmap='hot')

In [ ]:
training = build_training()

def fully_connected_layers(num_layers):
    
    connected_layers =[{'weights':tf.Variable(tf.random_normal([shape])),
                       'biases' :tf.Variable(tf.random_normal( shape )),
                        ''} for i in range(num_layers)]
    
    for i in range(num_layers): 
        connected_layer_ = 
    
    

In [ ]:
plt.imshow(tf.to_float(output_2[2]),cmap='hot')

In [ ]:
plt.imshow(training[2, :, :,:].astype(np.float32),cmap='hot')

In [ ]:
#plt.imshow(output_1[2, :, :,:].astype(np.float32),cmap='hot')
plt.imshow(result_1[2].astype(np.float32).reshape(400,400),cmap='hot')